In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd

from scipy.fft import fft

In [3]:
data = pd.read_csv("../data/combined.csv").set_index("date")
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1556 entries, 1993-04-05 to 2023-01-23
Data columns (total 74 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   inflation_one_month                         1556 non-null   float64
 1   inflation_two_month                         1556 non-null   float64
 2   inflation_three_month                       1556 non-null   float64
 3   inflation_six_month                         1556 non-null   float64
 4   inflation_twelve_month                      1556 non-null   float64
 5   opec_price                                  1556 non-null   float64
 6   brent_price                                 1556 non-null   float64
 7   wti_price                                   1556 non-null   float64
 8   oil_reserves_africa                         1556 non-null   float64
 9   oil_reserves_africa_bp                      1556 non-null   float64
 10  oi

In [4]:
# fft with domain size of 52

FFT_DOMAIN = 52
FFT_FREQS = [0, 1, 3, 7, 12, 25, 51]

fft_accumulator = []


def hack(x):
    fft_accumulator.append(fft(x.values, norm="ortho"))

    return 0


dummy_fourier = data.rolling(window=FFT_DOMAIN).apply(hack)

num_cols = len(data.columns)
num_freq_blocks = len(fft_accumulator)
blocks_per_column = num_freq_blocks // num_cols

fft_accumulator = np.array(fft_accumulator)

dfs = []
for n, col in enumerate(data.columns):
    freq_block = fft_accumulator[n * blocks_per_column : (n + 1) * blocks_per_column][
        :, FFT_FREQS
    ]
    dfs.append(pd.DataFrame(freq_block, columns=[f"{col}_f{m}" for m in FFT_FREQS]))

fourier = pd.concat(dfs, axis=1, join="inner")
fourier

,inflation_one_month_f0,inflation_one_month_f1,inflation_one_month_f3,inflation_one_month_f7,inflation_one_month_f12,inflation_one_month_f25,inflation_one_month_f51,inflation_two_month_f0,inflation_two_month_f1,inflation_two_month_f3,...,oil_reserves_yemen_f12,oil_reserves_yemen_f25,oil_reserves_yemen_f51,gas_price_f0,gas_price_f1,gas_price_f3,gas_price_f7,gas_price_f12,gas_price_f25,gas_price_f51
0,1.830511-0.000000j,0.007644+0.299783j,0.048824+0.294946j,-0.106198+0.126905j,0.018646-0.027571j,0.016701+0.009595j,0.007644-0.299783j,3.536214-0.000000j,0.118925+0.578197j,0.386436+0.298109j,...,1.163419e+05+7.984664e+04j,1.623364e+04+9.307088e+04j,-1.772365e+06+3.465440e+04j,7.611873-0.000000j,-0.040589-0.133535j,-0.007157+0.023961j,0.014401-0.000181j,0.000848-0.006389j,0.008912-0.000552j,-0.040589+0.133535j
1,1.844378-0.000000j,-0.014780+0.300190j,-0.045971+0.298010j,-0.156216+0.015043j,0.031289+0.028953j,-0.031502-0.005840j,-0.014780-0.300190j,3.536214-0.000000j,0.048364+0.588317j,0.255614+0.415769j,...,-7.073795e+04+7.984664e+04j,1.793772e+04-9.593249e+04j,-1.800536e+06+2.535336e+05j,7.604108-0.000000j,-0.031906-0.138389j,-0.022450+0.017112j,0.004535+0.004847j,0.005509-0.007637j,-0.001071+0.000686j,-0.031906+0.138389j
2,1.830511-0.000000j,-0.064623+0.294549j,-0.161626+0.257425j,-0.124046-0.117334j,-0.026642+0.020785j,0.045742+0.000329j,-0.064623-0.294549j,3.536214-0.000000j,-0.022902+0.589857j,0.091569+0.479392j,...,-9.372520e+04-1.094703e+05j,4.262904e+04+9.146098e+04j,-1.805721e+06+4.746500e+05j,7.596203-0.000000j,-0.022840-0.142179j,-0.034450+0.005237j,-0.005862+0.000692j,0.007293-0.003299j,0.008828-0.001763j,-0.022840+0.142179j
3,1.816643-0.000000j,-0.113422+0.282940j,-0.255374+0.178466j,-0.003628-0.181036j,-0.025516-0.037709j,-0.031682+0.003516j,-0.113422-0.282940j,3.536214-0.000000j,-0.093835+0.582796j,-0.084376+0.480710j,...,9.100338e+04-1.587108e+05j,-8.689025e+02-9.202723e+04j,-1.787816e+06+6.952163e+05j,7.589131-0.000000j,-0.012556-0.144748j,-0.040681-0.009828j,-0.009095-0.009223j,0.003301-0.000179j,-0.001530+0.001962j,-0.012556+0.144748j
4,1.802776-0.000000j,-0.160466+0.265534j,-0.315030+0.071394j,0.123906-0.133145j,0.032687-0.043642j,0.044794-0.008980j,-0.160466-0.265534j,3.536214-0.000000j,-0.163399+0.567236j,-0.249355+0.419552j,...,1.617141e+05+1.513453e+04j,6.803009e+04+8.444279e+04j,-1.747057e+06+9.124536e+05j,7.582752-0.000000j,-0.001350-0.145975j,-0.040517-0.025877j,-0.003358-0.017699j,-0.000194-0.003077j,0.007615-0.002901j,-0.001350+0.145975j
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,3.244996-0.000000j,-0.249827-0.511908j,0.129240-0.552691j,-0.041313-0.113561j,0.001015+0.008770j,-0.020767+0.059282j,-0.249827+0.511908j,6.878282-0.000000j,-0.550819-1.057056j,-0.392713-0.633271j,...,-6.778187e-09-9.819899e-09j,1.428861e-09-7.797043e-09j,1.289005e-07+2.362189e-08j,29.290389-0.000000j,-2.207412-0.444636j,-0.794199+0.181286j,-0.181043-0.013669j,-0.107685-0.020546j,0.030086+0.013401j,-2.207412+0.444636j
1501,3.217261-0.000000j,-0.213834-0.541632j,0.290896-0.480781j,0.039214-0.126988j,-0.011927-0.025468j,0.041003-0.064696j,-0.213834+0.541632j,6.808945-0.000000j,-0.488221-1.124100j,-0.207464-0.755964j,...,-6.778187e-09-9.819899e-09j,1.428861e-09-7.797043e-09j,1.289005e-07+2.362189e-08j,29.266537-0.000000j,-2.161400-0.710343j,-0.829176-0.120580j,-0.125639-0.162430j,0.004542-0.133055j,-0.007804-0.012552j,-2.161400+0.710343j
1502,3.147924-0.000000j,-0.215821-0.571815j,0.377648-0.370973j,0.075076-0.106756j,0.015487-0.083742j,0.035926+0.060809j,-0.215821+0.571815j,6.698005-0.000000j,-0.459297-1.188125j,-0.029644-0.819747j,...,-6.778187e-09-9.819899e-09j,1.428861e-09-7.797043e-09j,1.289005e-07+2.362189e-08j,29.259603-0.000000j,-2.066902-0.966528j,-0.739018-0.409232j,0.033669-0.206943j,0.131796-0.018413j,0.016143+0.010684j,-2.066902+0.966528j
1503,3.078586-0.000000j,-0.214154-0.602018j,0.419824-0.237537j,0.083713-0.066497j,0.076640-0.063552j,0.025838-0.064393j,-0.214154+0.60

In [5]:
fourier.abs().mean()

inflation_one_month_f0     1.363540
inflation_one_month_f1     0.321532
inflation_one_month_f3     0.339968
inflation_one_month_f7     0.108490
inflation_one_month_f12    0.031693
                             ...   
gas_price_f3               0.238367
gas_price_f7               0.081744
gas_price_f12              0.050164
gas_price_f25              0.028699
gas_price_f51              0.817982
Length: 518, dtype: float64

In [9]:
# momentum analogous quantity
PRICE_COLS = ["wti_price", "brent_price", "opec_price", "gas_price"]
MOMENTUM_SPREADS = [1, 4, 13, 26, 52]

momentum = {}

for col in PRICE_COLS:
    for spread in MOMENTUM_SPREADS:
        momentum[f"{col}_p{spread}"] = data[col] / data[col].shift(spread)

momentum = pd.DataFrame(momentum).fillna(1).replace([np.inf], 1)
momentum

,wti_price_p1,wti_price_p4,wti_price_p13,wti_price_p26,wti_price_p52,brent_price_p1,brent_price_p4,brent_price_p13,brent_price_p26,brent_price_p52,opec_price_p1,opec_price_p4,opec_price_p13,opec_price_p26,opec_price_p52,gas_price_p1,gas_price_p4,gas_price_p13,gas_price_p26,gas_price_p52
date,,,,,,,,,,,,,,,,,,,,
1993-04-05,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1993-04-12,1.022000,1.000000,1.000000,1.000000,1.000000,1.016830,1.000000,1.000000,1.000000,1.000000,1.019521,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1993-04-19,0.996575,1.000000,1.000000,1.000000,1.000000,0.998932,1.000000,1.000000,1.000000,1.000000,0.997702,1.000000,1.000000,1.000000,1.000000,1.010300,1.000000,1.000000,1.000000,1.000000
1993-04-26,0.995582,1.000000,1.000000,1.000000,1.000000,0.997862,1.000000,1.000000,1.000000,1.000000,0.996673,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1993-05-03,0.983728,0.997500,1.000000,1.000000,1.000000,0.994108,1.007600,1.000000,1.000000,1.000000,0.988703,1.002343,1.000000,1.000000,1.000000,1.006487,1.016854,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,1.028196,0.894073,0.861073,0.645028,1.055353,1.028513,0.877403,0.883930,0.645987,1.095445,0.997319,0.906381,0.809218,0.673709,1.082433,0.964509,0.859421,0.857597,0.638373,0.952577
2023-01-02,1.028754,0.981458,0.925953,0.716352,1.078878,0.998515,0.933796,0.911227,0.690576,1.091598,1.052477,0.985735,0.915600,0.699532,1.062540,0.990414,0.877775,0.835856,0.643302,0.949037
2023-01-09,1.025233,0.999243,0.989386,0.714041,1.041815,1.014254,0.962366,0.937557,0.679990,1.046955,0.997811,1.030003,0.890493,0.718225,1.041508,1.039963,0.950628,0.852136,0.682722,0.985211


In [10]:
# inflation quadratics
inflation_contribution = {}
INFLATION_COLS = [col for col in data.columns if "inflation" in col]

for col in PRICE_COLS:
    for inflation in INFLATION_COLS:
        inflation_contribution[f"{col}_{inflation}_contribution"] = (
            data[col] * data[inflation] / 100.0
        )

inflation_contribution = pd.DataFrame(inflation_contribution)
inflation_contribution

,wti_price_inflation_one_month_contribution,wti_price_inflation_two_month_contribution,wti_price_inflation_three_month_contribution,wti_price_inflation_six_month_contribution,wti_price_inflation_twelve_month_contribution,brent_price_inflation_one_month_contribution,brent_price_inflation_two_month_contribution,brent_price_inflation_three_month_contribution,brent_price_inflation_six_month_contribution,brent_price_inflation_twelve_month_contribution,opec_price_inflation_one_month_contribution,opec_price_inflation_two_month_contribution,opec_price_inflation_three_month_contribution,opec_price_inflation_six_month_contribution,opec_price_inflation_twelve_month_contribution,gas_price_inflation_one_month_contribution,gas_price_inflation_two_month_contribution,gas_price_inflation_three_month_contribution,gas_price_inflation_six_month_contribution,gas_price_inflation_twelve_month_contribution
date,,,,,,,,,,,,,,,,,,,,
1993-04-05,0.08000,0.20000,0.30000,0.44000,0.68000,0.07368,0.18420,0.27630,0.40524,0.62628,0.07684,0.19210,0.28815,0.42262,0.653140,0.004272,0.010680,0.016020,0.023496,0.036312
1993-04-12,0.04088,0.12264,0.24528,0.36792,0.71540,0.03746,0.11238,0.22476,0.33714,0.65555,0.03917,0.11751,0.23502,0.35253,0.685475,0.002136,0.006408,0.012816,0.019224,0.037380
1993-04-19,0.04074,0.12222,0.24444,0.36666,0.71295,0.03742,0.11226,0.22452,0.33678,0.65485,0.03908,0.11724,0.23448,0.35172,0.683900,0.002158,0.006474,0.012948,0.019422,0.037765
1993-04-26,0.04056,0.12168,0.24336,0.36504,0.70980,0.03734,0.11202,0.22404,0.33606,0.65345,0.03895,0.11685,0.23370,0.35055,0.681625,0.002158,0.006474,0.012948,0.019422,0.037765
1993-05-03,0.03990,0.11970,0.23940,0.35910,0.69825,0.03712,0.11136,0.22272,0.33408,0.64960,0.03851,0.11553,0.23106,0.34659,0.673925,0.002172,0.006516,0.013032,0.019548,0.038010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,0.15024,0.22536,0.45072,1.35216,4.28184,0.16160,0.24240,0.48480,1.45440,4.60560,0.15626,0.23439,0.46878,1.40634,4.453410,0.006468,0.009702,0.019404,0.058212,0.184338
2023-01-02,0.15456,0.23184,0.46368,1.39104,4.40496,0.16136,0.24204,0.48408,1.45224,4.59876,0.16446,0.24669,0.49338,1.48014,4.687110,0.006406,0.009609,0.019218,0.057654,0.182571
2023-01-09,0.15846,0.23769,0.47538,1.42614,4.51611,0.16366,0.24549,0.49098,1.47294,4.66431,0.16410,0.24615,0.49230,1.47690,4.676850,0.006662,0.009993,0.019986,0.059958,0.189867
